<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Prompts_e_Ingenier%C3%ADa_de_Prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompts e Ingeniería de Prompts dentro de LangChain  
---
Este cuaderno se enfoca en los **Prompts** y en la **Ingeniería de Prompts** dentro de LangChain. Abordaremos los fundamentos de los prompts, por qué son tan importantes y cómo LangChain nos ayuda a crearlos y reutilizarlos de manera efectiva con distintas clases de Templates: `PromptTemplate`, `FewShotPromptTemplate` y `ChatPromptTemplate`.

<br>

## 1. Introducción a los Prompts



In [1]:
!pip install langchain -qU


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.7 MB/s eta 0:00:00


In [2]:
from langchain_core.prompts import PromptTemplate

#Instanciar usando .from_template es la forma preferida
plantilla ="Cuentame un chiste sobre {tema}"
prompt_template = PromptTemplate.from_template(plantilla)
prompt = prompt_template.format(tema="enanos")

prompt

'Cuentame un chiste sobre enanos'

In [9]:
# Para eviar el uso de algunas variables, se suele escribir de forma mas compacta
# en adelante usaremos esta forma.

prompt_template = PromptTemplate.from_template("Cuentame un chiste sobre {tema}")
prompt = prompt_template.format(tema="enanos")

prompt

'Cuentame un chiste sobre enanos'

In [3]:
#Instanciar usando .from_template es la forma preferida
prompt_template = PromptTemplate.from_template("Cuentame un chiste {adjetivo} sobre {tema}")
prompt = prompt_template.format(adjetivo="malo", tema="enanos")

prompt

'Cuentame un chiste malo sobre enanos'

In [12]:
# Usando el constructor de clase
prompt_template = PromptTemplate(
    input_variables=["adjetivo", "tema"],
    template="Cuentame un chiste {adjetivo} sobre {tema}")

prompt = prompt_template.format(adjetivo="malo", tema="enanos")

prompt


'Cuentame un chiste malo sobre enanos'

In [10]:
prompt_template = PromptTemplate.from_template('''
Proporciona un resumen conciso del siguiente tema: {tema}
Usa un nivel de lenguaje {nivel}
Incluye al menos los siguinetes puntos clave:
* Información de antecedentes
* Ideas principales
* Detalles de apoyo
* Importancia o impacto
Manten una actitud neutral y objetiva en tu resumen.
''')
prompt = prompt_template.format(tema="Cambio climatico", nivel="técnico")

prompt

'\nProporciona un resumen conciso del siguiente tema: Cambio climatico\nUsa un nivel de lenguaje técnico\nIncluye al menos los siguinetes puntos clave:\n* Información de antecedentes\n* Ideas principales\n* Detalles de apoyo\n* Importancia o impacto\nManten una actitud neutral y objetiva en tu resumen.\n'

In [11]:
diagnostico_prompt_template = PromptTemplate.from_template('''
Eres un médico informado y empático. Con base en los siguientes síntomas descritos por un paciente, proporciona un diagnóstico potencial y sugiere los próximos pasos para el tratamiento.
Síntomas del paciente: {síntomas}
''')

prompt=diagnostico_prompt_template.format(síntomas="Fiebre, dolor de garganta, tos seca")

prompt

'\nEres un médico informado y empático. Con base en los siguientes síntomas descritos por un paciente, proporciona un diagnóstico potencial y sugiere los próximos pasos para el tratamiento.\nSíntomas del paciente: Fiebre, dolor de garganta, tos seca\n'

In [15]:
consejo_financiero_prompt_template = PromptTemplate.from_template('''
Eres un asesor financiero certificado. En función del perfil financiero del usuario, brindar asesoramiento de inversión personalizado.
Perfil de usuario:
- Edad: {edad}
- Ingresos: {ingresos}
- Tolerancia al riesgo: {tolerancia_riesgo}
- Objetivos de inversión: {objetivos_inversion}
''')

prompt = consejo_financiero_prompt_template.format(edad=50, ingresos="100000€", tolerancia_riesgo="moderada", objetivos_inversion="ahorros jubilación" )

prompt

'\nEres un asesor financiero certificado. En función del perfil financiero del usuario, brindar asesoramiento de inversión personalizado.\nPerfil de usuario:\n- Edad: 50\n- Ingresos: 100000€\n- Tolerancia al riesgo: moderada\n- Objetivos de inversión: ahorros jubilación\n'

`FewShotPromptTemplate` en LangChain es una herramienta poderosa para el aprendizaje de pocos ejemplos (few-shot learning) en aplicaciones de lenguaje. Te permite definir un conjunto de ejemplos de entrada/salida y, junto con una plantilla de prompt, generar prompts para un modelo de lenguaje que imitan un comportamiento deseado. Básicamente, ayuda a entrenar a un modelo para que aprenda a partir de unos pocos ejemplos cómo realizar una tarea.

Aquí te explico su funcionamiento y te doy ejemplos:

**¿Cómo funciona `FewShotPromptTemplate`?**

1. **`examples`:** Un array de diccionarios. Cada diccionario representa un ejemplo y contiene pares clave-valor con las entradas y salidas deseadas para una tarea específica.
    
2. **`example_prompt`:** Una instancia de `PromptTemplate` que define cómo se formatea cada ejemplo individualmente. Define las variables de entrada que se utilizarán para cada ejemplo.
    
3. **`prefix`:** Una cadena que va al principio del prompt completo. Suele ser una instrucción general o una descripción de la tarea.
    
4. **`suffix`:** Una cadena que va al final del prompt completo. Normalmente aquí es donde se introduce la nueva entrada a procesar por el modelo.
    
5. **`input_variables`:** Una lista de nombres de variables que se utilizarán en el `suffix`.
    
6. **`example_separator`:** Una cadena que separa cada ejemplo en el prompt. Por defecto es `\n\n`.
    
7. **`template_format`:** Define el formato en el que se va a procesar la plantilla. Por defecto es `f-string`. Otras opciones son `jinja2`.
    

**En resumen, `FewShotPromptTemplate` toma los ejemplos, los formatea usando `example_prompt`, los une con `example_separator`, y los coloca entre `prefix` y `suffix` para crear el prompt final. Este prompt contiene el contexto necesario (los ejemplos) para que el modelo de lenguaje pueda inferir la tarea y aplicarla a la nueva entrada proporcionada en el `suffix`.**

Hay que usar la sintaxis con el constructor de clase. No hay disponible ningun metodo que simplifique la sintaxis.

In [19]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# Lista de ejemplo, donde cada ejemplo es un diccionario
ejemplos = [
    {"input": "Hello", "output": "Bonjour"},
    {"input": "Goodbye", "output": "Au revoir"},
    {"input": "Thank you", "output": "Merci"},
]

#Instancia de PromptTemplate para formatear los ejemplos
traduccion_prompt_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Creamos el prompt pasando:
# la lista de ejemplos
# el formateador de los ejemplos
# el prefix, que suele ser la descripcion general de la tarea
# el suffix, que donde introduciomes la pregunta con la variables
# La lista de variables


prompt = FewShotPromptTemplate(
    examples=ejemplos,
    example_prompt=traduccion_prompt_template,
    prefix="Translate the following English words to French:",
    suffix="Input: {english_word}\nOutput:",
    input_variables=["english_word"],
)

print(prompt.format(english_word="Car"))

Translate the following English words to French:

Input: Hello
Output: Bonjour

Input: Goodbye
Output: Au revoir

Input: Thank you
Output: Merci

Input: Car
Output:


Crear un prompt con ejemplos para que indique la capital de un pais

Refs:



Las prompt templatespermiten crear solicitudes dinámicas y flexibles mediante la incorporación de variables y opciones de formato, lo que le permite personalizar las solicitudes en función de los datos de entrada o tareas específicas.

- Son dinamicas: tinene placeholders para variables
- Pueden reutilizarse

Un prompt template como este "Explicame {tema} con nivel de dificultad {nivel}" es reutilizable y flexible, sirve para muchos situaciones.

"traduce {frase} del {idioma_entrada} a {idioma_salida}"


Diferencias entre ChatPromptTemplate y PrompTemplate
Difrenecias entre .format y .invoke

- Pueden tener locga condicional interna (if)
- Pueden anidarse

---
¿Qué es Prompt?  

Un mensaje para un modelo de lenguaje es un conjunto de instrucciones o entradas proporcionadas por un usuario para guiar la respuesta del modelo, ayudándolo a comprender el contexto y generar resultados relevantes y coherentes basados ​​en el lenguaje, como responder preguntas, completar oraciones o participar en una conversación.  

---
3 tipos de plantillas de indicaciones de LangChain¶
Cuando se solicita en LangChain, se recomienda (aunque no es obligatorio) utilizar una clase de plantilla predefinida como:

PromptTemplatepara crear indicaciones básicas.
FewShotPromptTemplatepara un aprendizaje de unos pocos disparos.
ChatPromptTemplatepara modelar interacciones de chatbot.